In [1]:
import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from utils import get_backend_properties_v1
from mlp import MLP1, MLP2, MLP3, encode_data

import matplotlib.pyplot as plt
import seaborn as sns
from noise_utils import AddNoise, RemoveReadoutErrors
from mlp import recursive_dict_loop, count_gates_by_rotation_angle
from mbd_utils import calc_imbalance
import torch
from torch import nn
from sklearn import datasets
import sklearn
from utils import circuit_to_graph_data_json
from utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
)
from qiskit.circuit.random import random_circuit
from sklearn.model_selection import train_test_split
import gate_dict

In [2]:
backend = FakeLima()
gate_dict = gate_dict.gate_dict()

In [3]:
def create_train_and_test_data(n_qubits: int, 
                               circuit_depth: int, 
                               pauli_terms: int, 
                               pauli_coeff: float = 1.0, 
                               max_entries: int = 10, 
                               properties = get_backend_properties_v1(backend)):
    
    circuits_list = []
    ideal_exp_vals_list = []
    noisy_exp_vals_list = []

    for _ in range(max_entries):
        qc = random_circuit(n_qubits, random.randint(1, circuit_depth))

        observable = generate_random_pauli_sum_op(n_qubits, pauli_terms, pauli_coeff)

        ideal_exp_val, noisy_exp_val = create_estimator_meas_data(
            backend=backend, circuit=qc, observable=observable
        )

        circuits_list.append(qc)
        ideal_exp_vals_list.append(ideal_exp_val)
        noisy_exp_vals_list.append(noisy_exp_val)

    return circuits_list, ideal_exp_vals_list, noisy_exp_vals_list


In [4]:
# circ specified 
n_qubits = 3
circ_depth = 5
pauli_terms = 1

In [5]:
train_circuits, train_ideal_exp_vals, train_noisy_exp_vals = create_train_and_test_data(n_qubits=n_qubits,
                                                                                        circuit_depth=circ_depth,
                                                                                        pauli_terms=pauli_terms,
                                                                                        max_entries=200)

test_circuits, test_ideal_exp_vals, test_noisy_exp_vals = create_train_and_test_data(n_qubits=n_qubits,
                                                                                    circuit_depth=circ_depth,
                                                                                    pauli_terms=pauli_terms,
                                                                                    max_entries=40)

In [6]:
def get_qc_operations(qc):
    qc = transpile(circuits=qc, 
               backend=backend, 
               optimization_level=0)
    operations = []
    for operation in qc.data:
        operations.append(operation)
    return operations

In [7]:
def operations_to_features(circuits, exp_vals, noisy_exp_vals, n_qubits):
    X = []
    i = 0
    for qc in circuits:
        circuit_instructions = get_qc_operations(qc)
        # CircuitInstruction(operation=Instruction(name='rz', num_qu .... :)))))))
        features = []
        for i in range(n_qubits):
            features.append([])
        for circuit_instruction in circuit_instructions:
            gate_instruction = circuit_instruction.operation # the quantum gate applied to a qubit/qubits
            op_name = gate_instruction.name
            op_params = gate_instruction.params

            if len(op_params) == 0:
                op_params = float('0')
            else:
                op_params = op_params[0]

            op_encoded = gate_dict[op_name]
            op_qubit = circuit_instruction.qubits
            op_qubit = op_qubit[0].index

            qubit_feature = [op_encoded, op_params]
            features[op_qubit].append(qubit_feature)
            #print(f'{op_encoded} | {op_params} | {op_qubit}')

        max_length = max(len(sublist) for sublist in features)
        for sublist in features:
            while len(sublist) < max_length:
                sublist.append([float('0'), float('0')])

        # features.append(noisy_exp_vals[i])
        # i += 1

        X.append(features)

        

    y = []
    for exp_val in exp_vals:
        y.append(exp_val)
        
    return X, y

In [8]:
X_train, y_train = operations_to_features(train_circuits, train_ideal_exp_vals, train_noisy_exp_vals, n_qubits)
X_test, y_test = operations_to_features(test_circuits, test_ideal_exp_vals, test_noisy_exp_vals, n_qubits)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

In [10]:
import numpy as np

def evaluate_model(rnn_extractor, predictor, criterion, X, y):
    with torch.no_grad():
        all_predictions = []
        all_losses = []
        for i in range(len(X)):
            circ_features_tensor = torch.FloatTensor(X[i])
            rnn_output = rnn_extractor(circ_features_tensor)
            pred = predictor(rnn_output)
            loss = criterion(pred, torch.FloatTensor([y[i]]))
            
            all_predictions.append(pred.item())
            all_losses.append(loss.item())
        
        mse = np.mean(np.square(np.array(all_predictions) - np.array(y)))
        rmse = np.sqrt(mse)
        
        return rmse


In [21]:
class RNNFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNFeatureExtractor, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        
    def forward(self, x):
        #print(x.shape)
        output, _ = self.rnn(x)
        #print(output[:, -1, :].shape)
        return output[:, -1, :] 

class ExpectationValuePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ExpectationValuePredictor, self).__init__()
        self.fc = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size) 
        
    def forward(self, x):
        #print(x.shape)
        x = self.fc(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        output = 0
        for i in range(n_qubits):
            output += x[i]
        output = output/n_qubits
        return output

def train_and_evaluate_model(rnn_extractor, predictor, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs):
    for epoch in range(num_epochs):
        rnn_extractor.train()
        predictor.train()
        train_losses = []
        for i in range(len(X_train)):
            circ_features_tensor = torch.FloatTensor(X_train[i])
            rnn_output = rnn_extractor(circ_features_tensor)
            pred = predictor(rnn_output)
            loss = criterion(pred, torch.FloatTensor([y_train[i]]))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())

            #print(f'{predictor.parameters()}    |    {rnn_extractor.parameters()}')
        
        train_loss = np.mean(train_losses)
        
        # Evaluation phase
        rnn_extractor.eval()
        predictor.eval()
        test_mse = evaluate_model(rnn_extractor, predictor, criterion, X_test, y_test)

        if epoch % 1 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                f"Train Loss: {train_loss:.4f}, "
                f"Test RMSE: {test_mse:.4f}")
            
input_size = 2
hidden_size = 8
num_layers = 8
output_size = 1


rnn_extractor = RNNFeatureExtractor(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
predictor = ExpectationValuePredictor(input_size=hidden_size, hidden_size=32, output_size=output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(list(predictor.parameters()) + list(rnn_extractor.parameters()), lr=0.01)

num_epochs = 10
train_and_evaluate_model(rnn_extractor, predictor, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs)


Epoch 1/10, Train Loss: 0.1248, Test RMSE: 0.3582
Epoch 2/10, Train Loss: 0.1236, Test RMSE: 0.3582
Epoch 3/10, Train Loss: 0.1232, Test RMSE: 0.3575
Epoch 4/10, Train Loss: 0.1222, Test RMSE: 0.3576
Epoch 5/10, Train Loss: 0.1207, Test RMSE: 0.3589
Epoch 6/10, Train Loss: 0.1195, Test RMSE: 0.3586
Epoch 7/10, Train Loss: 0.1181, Test RMSE: 0.3606
Epoch 8/10, Train Loss: 0.1178, Test RMSE: 0.3609
Epoch 9/10, Train Loss: 0.1173, Test RMSE: 0.3624
Epoch 10/10, Train Loss: 0.1170, Test RMSE: 0.3630


In [22]:
train_and_evaluate_model(rnn_extractor, predictor, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs)

Epoch 1/10, Train Loss: 0.1167, Test RMSE: 0.3633
Epoch 2/10, Train Loss: 0.1165, Test RMSE: 0.3632
Epoch 3/10, Train Loss: 0.1163, Test RMSE: 0.3631
Epoch 4/10, Train Loss: 0.1159, Test RMSE: 0.3631
Epoch 5/10, Train Loss: 0.1156, Test RMSE: 0.3640
Epoch 6/10, Train Loss: 0.1155, Test RMSE: 0.3659
Epoch 7/10, Train Loss: 0.1142, Test RMSE: 0.3787
Epoch 8/10, Train Loss: 0.1145, Test RMSE: 0.3754
Epoch 9/10, Train Loss: 0.1131, Test RMSE: 0.3688
Epoch 10/10, Train Loss: 0.1165, Test RMSE: 0.3666


In [13]:
def fix_random_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [14]:
properties = get_backend_properties_v1(backend=backend)

In [15]:
def encode_data(circuits, properties, ideal_exp_vals, noisy_exp_vals, num_qubits, meas_bases=None):
    # if isinstance(noisy_exp_vals[0], list) and len(noisy_exp_vals[0]) == 1:
    #     noisy_exp_vals = [x[0] for x in noisy_exp_vals]

    gates_set = sorted(properties['gates_set'])     # must sort!

    if meas_bases is None:
        meas_bases = [[]]

    vec = [np.mean(recursive_dict_loop(properties, out=[], target_key1='cx', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='id', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='sx', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='x', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='rz', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='readout_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='t1'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='t2'))]
    vec = torch.tensor(vec) * 100  # put it in the same order of magnitude as the expectation values

    bin_size = 0.1 * np.pi
    num_angle_bins = int(np.ceil(4 * np.pi / bin_size))

    X = torch.zeros([len(circuits), len(vec) + len(gates_set) + num_angle_bins + num_qubits + len(meas_bases[0])])

    vec_slice = slice(0, len(vec))
    gate_counts_slice = slice(len(vec), len(vec)+len(gates_set))
    angle_bins_slice = slice(len(vec)+len(gates_set), len(vec)+len(gates_set)+num_angle_bins)
    exp_val_slice = slice(len(vec)+len(gates_set)+num_angle_bins, len(vec)+len(gates_set)+num_angle_bins+num_qubits)
    meas_basis_slice = slice(len(vec)+len(gates_set)+num_angle_bins+num_qubits, len(X[0]))

    X[:, vec_slice] = vec[None, :]

    for i, circ in enumerate(circuits):
        gate_counts_all = circ.count_ops()
        X[i, gate_counts_slice] = torch.tensor(
            [gate_counts_all.get(key, 0) for key in gates_set]
        ) * 0.01  # put it in the same order of magnitude as the expectation values

    for i, circ in enumerate(circuits):
        gate_counts = count_gates_by_rotation_angle(circ, bin_size)
        X[i, angle_bins_slice] = torch.tensor(gate_counts) * 0.01  # put it in the same order of magnitude as the expectation values

        # if num_qubits > 1: assert len(noisy_exp_vals[i]) == num_qubits
        # elif num_qubits == 1: assert isinstance(noisy_exp_vals[i], float)

        X[i, exp_val_slice] = torch.tensor(noisy_exp_vals[i])

    if meas_bases != [[]]:
        assert len(meas_bases) == len(circuits)
        for i, basis in enumerate(meas_bases):
            X[i, meas_basis_slice] = torch.tensor(basis)

    y = torch.tensor(ideal_exp_vals, dtype=torch.float32)

    return X, y

In [16]:
X_train_rf, y_train_rf = encode_data(train_circuits, properties, train_ideal_exp_vals, train_noisy_exp_vals, num_qubits=n_qubits)
X_test_rf, y_test_rf = encode_data(test_circuits, properties, test_ideal_exp_vals, test_noisy_exp_vals, num_qubits=n_qubits)
BATCH_SIZE = 32
fix_random_seed(42)
train_dataset = TensorDataset(torch.Tensor(X_train_rf), torch.Tensor(y_train_rf))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = TensorDataset(torch.Tensor(X_test_rf), torch.Tensor(y_test_rf))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE*1000, shuffle=False)

random seed fixed to 42


In [17]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
rfr = RandomForestRegressor(n_estimators=300)
rfr.fit(X_train_rf, y_train_rf)

RandomForestRegressor(n_estimators=300)

In [18]:
from sklearn.metrics import root_mean_squared_error

In [19]:
y_pred_rf = rfr.predict(X_test_rf)
rms = root_mean_squared_error(y_test_rf, y_pred_rf)

In [20]:
rms

0.041845341859475055